### test ff
Powered by [Eleonora Priori](https://www.ecoaz.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo) and [Pietro Terna](https://terna.to.it/) 


The trick above, useful to avoid scrolling output windows, does not work with recent jupyter versions; use settings.

In [1]:
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from typing import Tuple, List, Dict
import numpy as np
import pandas as pd
import pickle
import csv
import os
import sys

comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)



#generate random seed
repast4py.random.init(rng_seed=12345)
rng = repast4py.random.default_rng 

In [2]:
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, sector: int, labor: int, capital: float, capitalR: float, wage: float): 
                 #, minOrderDuration:int,\
                 #maxOrderDuration:int, recipe: float, laborProductivity: float, maxOrderProduction: float,\
                 #assetsUsefulLife: float, plannedMarkup: float, orderObservationFrequency: int, productionType: int,\
                 #sectorialClass: int):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid

        self.sector=sector
        self.labor=labor
        self.capital=capital
        self.capitalR=capitalR
        self.wage=wage
    
        """
        self.unavailableLabor=0
        self.unavailableCapital=0
        self.minOrderDuration=minOrderDuration
        self.maxOrderDuration=maxOrderDuration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.maxOrderProduction=maxOrderProduction
        self.assetsUsefulLife=assetsUsefulLife
        self.plannedMarkup=plannedMarkup
        self.orderObservationFrequency=orderObservationFrequency
        self.productionType=productionType
        self.sectorialClass=sectorialClass 
        """




class Model:
    
    def __init__(self):
        
      
        runner.schedule_repeating_event(0.0,  1, self.interactingWithFirms)

        runner.schedule_stop(0)
        runner.schedule_end_event(self.finish)
        
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################

        fileName = "ff_with_class_limits.csv" #input("file name? ")
        simulationFirmsExAnteNumber = int(input("how many firms? "))
        
        self.smart_capital = 2
        while self.smart_capital != 0 and self.smart_capital != 1:
            self.smart_capital = int(input("use smart capital? (Please, use only 1/0 to express True/False values) "))
            if self.smart_capital != 0 and self.smart_capital != 1: print("Please, use only 1/0 to express True/False values")
                   
        
        #importing csv file containing info about firms 
        with open(fileName, newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',')#, quoting=csv.QUOTE_NONNUMERIC)
            
            self.rowNumber=-1 #to skip the row of the headers
            k=0
            #for each record in .csv
            
            for row in firmReader:
                #print(row)
                labor = 0 
                if self.rowNumber>=0:
                        
                    sector = row[0]
                        
                    if row[4]=='': row[4]=0 # it pertains with last rows (naio sector 65)
                    for i in range(int(float(row[4]) * simulationFirmsExAnteNumber)// rankNum):
                        if row[0] == '1' or row[0] == '2' or row[0] == '3': #agri, silvi and fishing
                            randomizer = rng.uniform()
                            if randomizer <= 0.2: labor = 0
                            elif randomizer <= 0.9 : labor = 1
                            else: labor = 2
                        else:
                            if int(row[7]) == 0: row[7] = '1' #to avoid firms with 0 workers in the 0-9 class
                            labor= rng.integers(int(row[7]), int(row[8])+1) #ptptpt because integers exclude extremes                         
                            # at https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.integers.html
                            # random.Generator.integers(low, high=None,...
                            # high, if provided, one above the largest (signed) integer to be drawn from the distribution
                            
                        capital= float(row[11]) + rng.random()*(float(row[12]) - float(row[11]))
                        if self.smart_capital:
                            capital = labor * float(row[10]) # row[10] = recipe 
                            
                        capitalR = float(row[13])
                        wage = float(row[14])
                        
                        """
                        minOrderDuration= row[5]
                        maxOrderDuration= row[6]
                        recipe= row[7] #K/L 
                        laborProductivity= row[8]
                        maxOrderProduction= row[9]
                        avgAssetsUsefulLife=row[10]  #https://www.oecd.org/sdd/productivity-stats/43734711.pdf
                        plannedMarkup=row[11]
                        orderObservationFrequency=rng.integers(row[12], row[13]+1)
                        productionType=int(row[14]) #productionType in firm-features.csv indicates the production of
                                                #investment goods if it is into the investmentGoods list in yaml
                        sectorialClass=int(self.rowNumber)
                        """
                        
                        aFirm =Firm(k, rank, sector, labor, capital, capitalR, wage) #, minOrderDuration, maxOrderDuration, recipe, laborProductivity,\
                                #maxOrderProduction, avgAssetsUsefulLife, plannedMarkup, orderObservationFrequency, productionType,\
                                #sectorialClass)
                        context.add(aFirm)
                        k += 1 # first element of the UID of the agents
                self.rowNumber += 1
                self.simulationFirmsExPostNumber=k #one more, here is a count, not an id
        
    #interactingWithFirms
    def interactingWithFirms(self):

        #print(self.firmCount)

        if self.simulationFirmsExPostNumber==0:
            print("No firms created.")
        else:
            # a check
            countActualExPostFirmNumber = len(list(context.agents(agent_type=0)))
            if countActualExPostFirmNumber != self.simulationFirmsExPostNumber:
                print("DISASTER")
                quit()

            AFF_FirmNumber = 10000000
            Eu_FirmNumber = 20789413
            AFF_WorkerNumber = 9000000
            Eu_WorkerNumber = 152533008+9000000
            Eu_EmployeeCompensations = 7447036.79
            Eu_GDP_withVAT2022 = 16144780 #https://ec.europa.eu/eurostat/databrowser/view/tec00001/default/table?lang=en
            Eu_AddedValue2022 = 14303899 #naio table 2022 (milions) total Added value, gross
            
            propFactor = Eu_FirmNumber / countActualExPostFirmNumber

            countWorkers     =0
            countEmployeeCompensations=0
            countCapitalR = 0
            countAFF_Firms   =0
            countAFF_Workers =0
            for aFirm in context.agents(agent_type=0):
                countWorkers+=aFirm.labor
                countEmployeeCompensations+=aFirm.labor*aFirm.wage
                countCapitalR+=aFirm.capital*aFirm.capitalR
                if int(aFirm.sector) <= 3: 
                    countAFF_Firms+=1
                    countAFF_Workers+=aFirm.labor

            print()
            table_data = [
                [' ',          'EU 27', 'simulation', 'simulation to'],
                [' ',          ' ',     ' ',          'EU scale'],
                ['AFF Firm Number', AFF_FirmNumber, countAFF_Firms, int(countAFF_Firms*propFactor)], 
                ['AFF Worker Number', AFF_WorkerNumber, countAFF_Workers, int(countAFF_Workers*propFactor)], 
                ['Eu Firm Number', Eu_FirmNumber, countActualExPostFirmNumber, int(countActualExPostFirmNumber*propFactor)],
                ['Eu Worker Number', Eu_WorkerNumber, countWorkers, int(countWorkers*propFactor)],
                ['Eu Empl. Compens.', int(Eu_EmployeeCompensations), int(countEmployeeCompensations/1000000), 
                                                                int(countEmployeeCompensations*propFactor/1000000)],                                                                                                                              
                ['Eu Capital R', int(Eu_AddedValue2022-Eu_EmployeeCompensations), int(countCapitalR/1000), 
                                                                int(countCapitalR*propFactor/1000)],
                ['Eu GDP VAT', Eu_GDP_withVAT2022, ' ',' '],
                ['Eu A. Val. curr. p.', Eu_AddedValue2022, ' ',int(countEmployeeCompensations*propFactor/1000000)+
                                                                 int(countCapitalR*propFactor/1000)]                
            ]
            for row in table_data:
                print("{: >20} {: >20} {: >20} {: >20}".format(*row))

            print("\n AFF => Agriculture, Forestry and Fishing")
                


    #finish
    def finish(self):
        print("\nConcluded")
    
    def start(self):
        runner.execute()

In [3]:
def run():

    model = Model() 
    model.start()
    
run()

#file name: ff_with_class_limits.csv

how many firms?  1000000
use smart capital? (Please, use only 1/0 to express True/False values)  0



                                    EU 27           simulation        simulation to
                                                                           EU scale
     AFF Firm Number             10000000               481010             10003666
   AFF Worker Number              9000000               432360              8991882
      Eu Firm Number             20789413               999625             20789413
    Eu Worker Number            161533008              7707579            160296154
   Eu Empl. Compens.              7447036               334618              6959125
        Eu Capital R              6856862               200324              4166190
          Eu GDP VAT             16144780                                          
 Eu A. Val. curr. p.             14303899                                  11125315

 AFF => Agriculture, Forestry and Fishing

Concluded
